In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from numpy import array

## Check return sequences output in LSTM

In [6]:
#define the model
inputs1 = Input(shape=(3,1))
lstm1   = LSTM(1)(inputs1)
model   = Model(inputs = inputs1, outputs = lstm1)
#define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
#make and show prediction
print(model.predict(data))

[[-0.07278257]]


The above model gives single hidden state for three time steps. We can obtain the hidden state for each time step.

In [7]:
#define the model
inputs1 = Input(shape=(3,1))
lstm1   = LSTM(1, return_sequences=True)(inputs1)
model   = Model(inputs = inputs1, outputs = lstm1)
#define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
#make and show prediction
print(model.predict(data))

[[[0.01146479]
  [0.0311129 ]
  [0.05601001]]]


## Return States

In [8]:
#define the model
inputs1 = Input(shape=(3,1))
lstm1, state_h, state_c   = LSTM(1, return_state=True)(inputs1)
model   = Model(inputs = inputs1, outputs = [lstm1, state_h, state_c])
#define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
#make and show prediction
print(model.predict(data))

[array([[0.02113167]], dtype=float32), array([[0.02113167]], dtype=float32), array([[0.04163989]], dtype=float32)]


## Return States and sequences

In [9]:
#define the model
inputs1 = Input(shape=(3,1))
lstm1, state_h, state_c   = LSTM(1, return_state=True, return_sequences=True)(inputs1)
model   = Model(inputs = inputs1, outputs = [lstm1, state_h, state_c])
#define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
#make and show prediction
print(model.predict(data))

[array([[[-0.02711373],
        [-0.07580496],
        [-0.13912186]]], dtype=float32), array([[-0.13912186]], dtype=float32), array([[-0.26157045]], dtype=float32)]


# Encoder Decoder Example

In [13]:
def define_models(n_input, n_output, n_units):

    #define training encoder
    encoder_inputs = Input(shape = (None, n_input))
    encoder = LSTM(n_untis, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    #define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
    decoder_dense = Dense(n_output, activatin = "softmax")
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    #define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    #define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    #return all models
    return model, encoder_model, decoder_model

In [14]:
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)

# Scalable Sequence to Sequence problem

In [ ]:
n_features = 50 + 1

#generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique -1) for _ in range(length)]

def get_dataset(n_in, n_out, cardinality, n_samples):
    pass